In [ ]:
%pylab inline

In [ ]:
import numpy as np
import pylab as plt
import h5py
import astra
import cv2
import json
from pprint import pprint

In [ ]:
data_file = './8b3c11c2-371c-48a6-85ad-972b33151b43.h5'

In [ ]:
with h5py.File(data_file,'r') as h5f:
    empty_images_count = len(h5f['empty'])
    empty_images = None
    empty_file_number = 0
    for k,v in h5f['empty'].iteritems():
        if empty_images is None:
            empty_images = np.zeros(shape=(empty_images_count, v.shape[0], v.shape[1]), dtype='float32')
        attributes = json.loads(v.attrs.items()[0][1])[0]
        exposure = attributes['frame']['image_data']['exposure']
        empty_images[empty_file_number] = v.value.astype('float32') / exposure
        empty_file_number = empty_file_number + 1
    
    dark_images_count = len(h5f['dark'])
    dark_images = None
    dark_file_number = 0
    for k,v in h5f['dark'].iteritems():
        if dark_images is None:
            dark_images = np.zeros(shape=(dark_images_count, v.shape[0], v.shape[1]), dtype='float32')
        
        attributes = json.loads(v.attrs.items()[0][1])[0]
        exposure = int(attributes['frame']['image_data']['exposure'])
        dark_images[dark_file_number] = v.value.astype('float32') / exposure
        dark_file_number = dark_file_number + 1

In [ ]:
empty_image = empty_images.sum(axis=0)/len(empty_images)
dark_image = dark_images.sum(axis=0)/len(empty_images)

%xdel empty_images
%xdel dark_images

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(221)
plt.imshow(empty_image, vmin=0)
plt.colorbar(orientation='horizontal')

plt.subplot(222)
plt.imshow(dark_image, vmin=0)
plt.colorbar(orientation='horizontal')

empty_beam = empty_image-dark_image
plt.subplot(223)
plt.imshow(empty_beam, vmin=0)
plt.colorbar(orientation='horizontal')

mask = empty_beam > 0.005
plt.subplot(224)
plt.imshow(mask)
plt.colorbar(orientation='horizontal')
#TODO: Check thresshold automaticaly
# plt.plot([sum(empty_beam > i) for i in np.arange(100)])

In [ ]:
x_range = np.arange(0,0.2,0.01)
plt.plot(x_range, [sum(empty_beam > x).astype('float32')/np.prod(empty_beam.shape) for x in x_range])

In [ ]:
threshold = 100
mask_x_profile = mask.sum(axis=1)
mask_x_min = np.argwhere(mask_x_profile>threshold).min()+50
mask_x_max = np.argwhere(mask_x_profile>threshold).max()-50
print mask_x_min, mask_x_max

mask_y_profile = mask.sum(axis=0)
mask_y_min = np.argwhere(mask_y_profile>threshold).min()+50
mask_y_max = np.argwhere(mask_y_profile>threshold).max()-50
print mask_y_min, mask_y_max

plt.imshow(mask[mask_x_min:mask_x_max,mask_y_min:mask_y_max])

In [ ]:
mask = mask[mask_x_min:mask_x_max,mask_y_min:mask_y_max]
empty_beam = empty_beam[mask_x_min:mask_x_max,mask_y_min:mask_y_max]
dark_image = dark_image[mask_x_min:mask_x_max,mask_y_min:mask_y_max]

plt.imshow(empty_beam)
plt.colorbar(orientation='horizontal')

In [ ]:
#TODO: добавить поддержку, когда много кадров на одном угле
with h5py.File(data_file,'r') as h5f:
    data_images_count = len(h5f['data'])
    data_images = None
    data_file_number = 0
    data_angles = None
    for k,v in h5f['data'].iteritems():
        if data_images is None:
            data_images = np.zeros(shape=(data_images_count,
                                          mask_x_max-mask_x_min,
                                          mask_y_max-mask_y_min),
                                   dtype='float32')

        if data_angles is None:
            data_angles = np.zeros(shape=(data_images_count,), dtype='float32')

        attributes = json.loads(v.attrs.items()[0][1])[0]
        exposure = attributes['frame']['image_data']['exposure']
        data_angles[data_file_number] = attributes['frame']['object']['angle position']
        data_images[data_file_number] = v[mask_x_min:mask_x_max,mask_y_min:mask_y_max].astype('float32') / exposure

        data_file_number = data_file_number + 1

In [ ]:
data_beam = data_images - dark_image
data_beam[data_beam <= 0] = 1e-6
empty_beam[empty_beam <= 0] = 1e-6
%xdel data_images

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cv2.medianBlur(data_beam[5],3))
plt.colorbar(orientation='horizontal')

In [ ]:
data_beam/=empty_beam

In [ ]:
data_beam*=mask
data_beam[data_beam>1] = 1
data_beam[data_beam<1e-5] = 1

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(-np.log(data_beam[50]))
plt.colorbar()